# Lesson 12a: Class inheritance, local imports

In [1]:
# This code cell loads a PEP8 linter
# Linting is the process of flagging programming errors,
# bugs, stylistic errors, and other code problems
# pycodestyle is a linter that highlights any syntax that
# is not PEP8-compliant.

%load_ext pycodestyle_magic
%pycodestyle_on

## Class Inheritance

While `class`es are really useful, we will find ourselves repeating a lot of code if we don’t think through and plan the code in advance. Let’s try writing a simple chess game. Before we start writing any code, it helps immensely to think about what we will need first.

At the least, we will need:

- a chess board
- chess pieces
  - king
  - queen
  - rook
  - bishop
  - knight
  - pawn
  
The pieces have one common characteristics:

- a colour (white or black)

6 different types of pieces. That means repeating our `__init__()`, `__repr__()`, and other methods _6 times_! And if we change our minds and decide to change one of those features, we will have to change it in 6 places … Is there any way we can reduce that repetition?

Of course. Python lets us define classes _based on other classes_.

Since all chess pieces have some common characteristics, let’s define a `BasePiece` class that represents a _generic_ chess piece. It will implement code that is common to all chess pieces:

In [2]:
class BasePiece:
    def __init__(self, colour):
        if type(colour) != str:
            raise TypeError('colour argument must be str')
        elif colour.lower() not in {'white', 'black'}:
            raise ValueError('colour must be {white, black}')
        else:
            self.colour = colour

    def __repr__(self):
        return f'BasePiece({repr(self.colour)})'

## Child classes

Our child classes should have the attributes of the `BasePiece`, which is the parent class. We call them child classes because they are derived from the parent class. We say that the child class **inherits** the attributes and methods of the parent class.

Let’s start with our first child class, the `King`:

In [3]:
class King(BasePiece):
    pass

This child class inherits the `colour` attribute, as well as the `__init__()` and `__repr__()` methods from `BasePiece`.

In [4]:
k = King('white')
k

BasePiece('white')

## Class attributes

Hmm, that’s not so helpful now. Our `BasePiece` only needed a `colour` since it was a generic piece, but now that we are creating pieces of a specific type, `__repr__()` should also return the piece name.

We will need to **override** the `__repr__()` method of `BasePiece` by defining a new one for `King`. But how do we create a `name` attribute for `King` without going through the `__init__()` method?

We can set it as a **class attribute** instead.

Notice that the `colour` attribute of `BasePiece` was set only in `__init__()`. The `BasePiece` class **does not** actually have this attribute, only its instances have it:

In [5]:
b = BasePiece('white')
print(f'b.colour: {b.colour}')
print(f'BasePiece.colour: {BasePiece.colour}')

b.colour: white


AttributeError: type object 'BasePiece' has no attribute 'colour'

The `King` class, on the other hand, is a `king` piece, whether it has been instantiated or not. I should be able to do this:

    >>> King.name
    'king'
    
So let’s go ahead and give the `King` class a `name` class attribute, and a new `__repr__()` method.

In [6]:
class King(BasePiece):
    # define a `name` class attribute for the King class
    name = 'king'

    def __repr__(self):
        # define the __repr__() method for the King class
        ### BEGIN SOLUTION
        return f'King({repr(self.colour)})'
        ### END SOLUTION


k = King('white')
print(k)

7:9: E266 too many leading '#' for block comment
9:9: E266 too many leading '#' for block comment


King('white')


In [7]:
# AUTOGRADING: test class attribute and __repr__()
assert King.name == 'king', \
    '`name` attribute wrongly defined for King class'
assert repr(King('white')) == "King('white')", \
    "__repr__() method wrongly defined for King class"

Later, we are going to need a `__str__()` method to produce a simple description (e.g. `'white king'`) too. That’s a simple combination of the `colour` and `name` attributes, and it would be tedious to repeat the `__str__()` definition for all the piece classes.

So let’s define it in `BasePiece` instead, using `try-except` to catch the `NameError` if the `name` attribute is not found:

In [8]:
class BasePiece:
    def __init__(self, colour):
        if type(colour) != str:
            raise TypeError('colour argument must be str')
        elif colour.lower() not in {'white', 'black'}:
            raise ValueError('colour must be {white, black}')
        else:
            self.colour = colour

    def __repr__(self):
        return f'BasePiece({repr(self.colour)})'

    def __str__(self):
        try:
            # define __str__() to return a simple description
            # e.g. 'white king', 'black queen'
            ### BEGIN SOLUTION
            return f'{self.colour} {self.name}'
            ### END SOLUTION
        except NameError:
            return f'{self.colour} piece'


class King(BasePiece):
    name = 'king'

    def __repr__(self):
        return f'King({repr(self.colour)})'


k = King('white')
print(k)

17:13: E266 too many leading '#' for block comment
19:13: E266 too many leading '#' for block comment


white king


In [ ]:
# Test cell to check your code
assert str(King('white')) == 'white king', \
    "__str__() method wrongly defined for King class"

There, `King` is working much better now.

## Instance attributes and attribute overriding

Wait, why is `self.name` able to be used when we didn’t set it in `__init__()`? That’s because class attributes are available to all its instances. If you set the `name` attribute of an instance, it will override its class instance. And if you delete the instance attribute, the class attribute will be used again:

In [9]:
k.name = 'da king'
# Uses instance attribute
print(f'After overriding class attribute: {k.name}')
del k.name
# Back to class attribute
print(f'After removing instance attribute: {k.name}')

After overriding class attribute: da king
After removing instance attribute: king


Lets go ahead and make `Board` first, before we come back to look at the other pieces.

## Making `Board`

Our chess board needs to have an 8×8 grid. It also needs to have a way to keep track of which piece is on which square of the grid. How should we go about doing this?

A newcomer might think of creating an 8-by-8 nested list, like this:

In [ ]:
board = []
for x in range(8):
    none_row = [None]*8
    board.append(none_row)
board[0][4] = King('black')
board[7][4] = King('white')
board

But then you are going to have a hard time tracking their positions; how are you going to find `King('black')` after it has moved?

    for x in range(8):
        for y in range(8):
            piece =  board[x][y]
            if piece.name == 'king' and piece.colour = 'black':
                <your code goes here>

That could work, but it’s so inefficient. You have 64 board positions, and 32 board pieces. How complex is your code going to have to be?

It would be easier instead to just store the positions of the pieces. Lets think about our ideal code. We would like to be able to get the positions of each chess piece this way:

    >>> b = Board() # instantiates a game board
    <-- Suppose our chess pieces have been placed on the board at this point -->
    >>> b.find('white king') # indexes start from 0
    7,4
    
It would also be nice if we could examine which piece was at a particular position:

    >>> b.examine(7,4)
    'white king'
    
It looks like each piece is **mapped** to a position. Sounds familiar? Perhaps we could use a `list` of `dict`s to map each chess piece to a position!

Oh man, there’s no easy way around this. We are going to have to write code to set the initial position of each piece. Before you run the cell below, time to define the other classes first so this will work:

In [10]:
# Define the other chess piece classes here:


### BEGIN SOLUTION
class Queen(BasePiece):
    name = 'queen'

    def __repr__(self):
        return f'Queen({repr(self.colour)})'


class Bishop(BasePiece):
    name = 'bishop'

    def __repr__(self):
        return f'Bishop({repr(self.colour)})'


class Knight(BasePiece):
    name = 'knight'

    def __repr__(self):
        return f'Knight({repr(self.colour)})'


class Rook(BasePiece):
    name = 'rook'

    def __repr__(self):
        return f'Rook({repr(self.colour)})'


class Pawn(BasePiece):
    name = 'pawn'

    def __repr__(self):
        return f'Pawn({repr(self.colour)})'
### END SOLUTION

4:1: E266 too many leading '#' for block comment
38:1: E266 too many leading '#' for block comment


In [21]:
# Run this cell to update `Board` and generate a playing field


def baserow(colour):
    '''
    Returns a list containing the base row pieces:
    Rook, Knight, Bishop, Queen, King, Bishop, Knight, Rook
    '''
    baserow = [None]*8
    for col, piece in enumerate((Rook, Knight, Bishop, Queen,
                                 King, Bishop, Knight, Rook)):
        baserow[col] = piece(colour)
    return baserow


class Board:
    def __init__(self):
        self.field = [None]*32
        idx = 0

        colour = 'black'
        y = 7
        for x, piece in enumerate(baserow(colour)):
            self.field[idx] = {'piece': piece, 'position': (x, y)}
            idx += 1
        y = 6
        for x in range(8):
            self.field[idx] = {'piece': Pawn(colour), 'position': (x, y)}
            idx += 1

        colour = 'white'
        y = 1
        for x in range(8):
            self.field[idx] = {'piece': Pawn(colour), 'position': (x, y)}
            idx += 1
        y = 0
        for x, piece in enumerate(baserow(colour)):
            self.field[idx] = {'piece': piece, 'position': (x, y)}
            idx += 1


b = Board()
b.field

[{'piece': Rook('black'), 'position': (0, 7)},
 {'piece': Knight('black'), 'position': (1, 7)},
 {'piece': Bishop('black'), 'position': (2, 7)},
 {'piece': Queen('black'), 'position': (3, 7)},
 {'piece': King('black'), 'position': (4, 7)},
 {'piece': Bishop('black'), 'position': (5, 7)},
 {'piece': Knight('black'), 'position': (6, 7)},
 {'piece': Rook('black'), 'position': (7, 7)},
 {'piece': Pawn('black'), 'position': (0, 6)},
 {'piece': Pawn('black'), 'position': (1, 6)},
 {'piece': Pawn('black'), 'position': (2, 6)},
 {'piece': Pawn('black'), 'position': (3, 6)},
 {'piece': Pawn('black'), 'position': (4, 6)},
 {'piece': Pawn('black'), 'position': (5, 6)},
 {'piece': Pawn('black'), 'position': (6, 6)},
 {'piece': Pawn('black'), 'position': (7, 6)},
 {'piece': Pawn('white'), 'position': (0, 1)},
 {'piece': Pawn('white'), 'position': (1, 1)},
 {'piece': Pawn('white'), 'position': (2, 1)},
 {'piece': Pawn('white'), 'position': (3, 1)},
 {'piece': Pawn('white'), 'position': (4, 1)},
 {'p

That’s all the _information_ we need about the pieces. But we will need more methods in the process of programming the chess game. We will continue that in **Lesson 10b**.

For now, we have quite a lot of code scattered across many cells. Let’s put them all into a single file so that it is easier to manage. In Python, this is known as making a **module**

## Task 1: Making a `chess` (single-file) module

Copy the latest code for `Board`, `BasePiece`, and each chess piece class (`King`,`Queen`,`Bishop`,`Knight`,`Rook`,`Pawn`) into `chess.py`, overriding the old definitions.

## local imports

Besides the standard Python libraries, you can also install other libraries. The most common way of doing this is through the <b>P</b>ackage <b>I</b>nstaller for <b>P</b>ython, also known as `pip`. It acts like an “app store” for Python, except it has python libraries instead of apps.

You can’t run `pip` on the school laptops as you don’t have administrator permissions, but on your own laptop you can do so. We will look at `pip` use once we begin on group-based projects. For now, let’s look at a related concern: how do you import a library you wrote yourself, but which is not available on `pip`?

## Importing from another Python file (`.py`) in the same directory

A library can be very simple; nothing more than another `.py` file containing functions and classes. It can also be very complex, consisting of multiple layers of files and directories, possibly even requiring installation.

We have just created a`chess` module, inside `chess.py`, in the same directory as this Jupyter Notebook.
Let’s import those objects into this notebook.

In [ ]:
from chess import Board, BasePiece, King, Queen, Bishop, Knight, Rook, Pawn

b = Board()
b.field

For modules with many more objects, it can get tedious to list every single class and function. In those cases, to keep the names clear (remember how hard it is to name things?), we simply import the module. The classes (and any functions) are available with the `module.class` (or `module.function`) syntax:

In [ ]:
import chess

b = chess.Board()
b.field

Very similar to your normal `import`s, right?

Python will first search in the directory to see if there is a file or library named `chess`. If it doesn’t exist, then Python will search in a list of directories to see if there are any libraries or modules named `chess`. This list of directories are known as the **system path**.

In [ ]:
import sys

print('Directories in system path:')
for path in sys.path:
    print(path)

If nothing named `chess` is found in any of these places, Python raises a `ModuleNotFoundError`. For example:

In [1]:
import chess2

ModuleNotFoundError: No module named 'chess2'

# Feedback and suggestions

Any feedback or suggestions for this assignment?